<div class="alert alert-info" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<font size="5"><b><center>YANDEX MUSIC: UNDERSTANDING AND PREPROCESSING BY TEAM 13</center></b></font>

# Содержание ноутбука

*В данном ноутбуке представлено знакомство с данными и первичная предобработка*

# Описание задачи

**В этом хакатоне предлагается разработать решение, которое:**

- может классифицировать треки по признаку кавер-некавер;
- связывать (группировать) каверы и исходный трек;
- находит исходный трек в цепочке каверов.

# Описание данных

**Разметка каверов**

- track_id - уникальный идентификатор трека;
- track_remake_type - метка, присвоенная редакторами. Может принимать значения ORIGINAL и COVER;
- original_track_id - уникальный идентификатор исходного трека.

**Метаинформация**

- track_id - уникальный идентификатор трека;
- dttm - первая дата появления информации о треке;
- title - название трека;
- language - язык исполнения;
- isrc - международный уникальный идентификатор трека;
- genres - жанры;
- duration - длительность трека;

**Текст песен**

- track_id - уникальный идентификатор трека;
- lyric_id - уникальный идентификатор текста;
- text - текст трека.

# Библиотеки

In [1]:
from collections import deque 
import json
import os
import random
import re
import requests


import numpy as np
import pandas as pd
from tqdm import tqdm, notebook

import warnings
warnings.filterwarnings("ignore")
notebook.tqdm().pandas()

0it [00:00, ?it/s]

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 1000)

# Знакомство с данными

In [3]:
cwd = os.getcwd()
path_to_covers = cwd + '/data/covers.json'
path_to_lyrics = cwd + '/data/lyrics.json'
path_to_meta = cwd + '/data/meta.json'

In [4]:
covers = pd.read_json(path_to_covers, lines=True)
lyrics = pd.read_json(path_to_lyrics, lines=True)
meta = pd.read_json(path_to_meta, lines=True)

## covers

In [5]:
covers.head()

,original_track_id,track_id,track_remake_type
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL
1,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL
2,cd89fef7ffdd490db800357f47722b20,cd89fef7ffdd490db800357f47722b20,ORIGINAL
3,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL
4,None,d6288499d0083cc34e60a077b7c4b3e1,COVER


In [6]:
covers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71597 entries, 0 to 71596
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_track_id  4821 non-null   object
 1   track_id           71597 non-null  object
 2   track_remake_type  71597 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


В колонке original_track_id есть пропуски. 
Как и предупреждали организаторы, не для всех треков известны идентификаторы исходных треков.

In [7]:
print('Количество строк дубликатов в covers -', covers.duplicated('track_id').sum())

Количество строк дубликатов в covers - 0


При первом знакомстве других отклонений не замечено.

## lyrics

In [8]:
lyrics.head()

,lyricId,text,track_id
0,a951f9504e89759e9d23039b7b17ec14,"Живу сейчас обломами, обломками не той любви\nПопытками не то любить, что нужно\nТеряю смысл, ну и пусть, невыносимой стала грусть\nИ в комнате, что с потолком мне чужда\n\nЯ б уплыла в океан, сшила б красный сарафан, и... И...\nЯ б забыла, что ты есть, я б не лезла в эту сеть\nЯ бы, я бы, я бы, я бы не была бы здесь\n\nВолосы пропахли дымом, вечер был довольно длинным...\n\nЖиву сейчас попытками, попытками не пытками\nНайти себя и что-нибудь родное\nИщу я что-то, где-то там, пью чёрный кофе по утрам\nИ не пойму - да что ж это такое?\n\nЯ б уплыла в океан, сшила б красный сарафан, и... И...\nЯ б забыла что ты есть, я б не лезла в эту сеть\nЯ бы, я бы, я бы, я бы не была бы здесь\n\nЯ б уплыла в океан, сшила б красный сарафан, и... И...\nЯ б забыла что ты есть, я б не лезла в эту сеть\nЯ бы, я бы, я бы, я бы не была бы здесь\n\nВолосы пропахли дымом, вечер был довольно длинным...",1c4b1230f937e4c548ff732523214dcd
1,0c749bc3f01eb8e6cf986fa14ccfc585,"Tell me your fable\nA fable\nTell me your fable\nTell me your fable\nTell me your fable\nA fable\nTell me your fable\nTalk to me, tenderly\nShow reality, fantasy\nWe'll bound together\nAll win in one feat\nTalk to me, tenderly\nShow reality, fantasy\nWe'll bound together\nAll win in one feat\nThe fable, the fable, ah\nThe fable, the fable\nTell me your fable\nTell me your fable\nA fable, a fable\nA fable that will never end\nAnd now, I dream\nDream, dream",0faea89b0d7d6235b5b74def72511bd8
2,e2c8830fbc86e5964478243099eec23a,"You're ashamed about all your fears and doubts\nAnd how I hurt you\nCan you make it back from the aftermath?\nAnd how I left you\nNobody wants to be alone\nWith the fear of letting go\nIf you could hear me say it's gonna be ok\nWould you be ok?\n\nAnd though I've gone away\nI still see what you're going through\nIt kills me everyday\nTo know I killed what meant most to you\nSo when you pass my grave\nLeave a rose for what might have been\nBut know that it's ok\nYou shed your fears and find love again\n\nFor better or worse, you're the one I never thought I'd hurt\nBut looking back on these dreams\nNothing is what it seems\nKnow that you wake up to better things\n\nAnd though I've gone away\nI still see what you're going through\nIt kills me everyday to know I killed what meant most to you\nSo when you pass my grave\nLeave a rose for what might have been\nBut know that it's ok to shed your fears and find love again\n\nI hear you say\nI don't know how and I don't know why\nBut there ...",9c6dc41d5ccd9968d07f055da5d8f741
3,e2c8830fbc86e5964478243099eec23a,"You're ashamed about all your fears and doubts\nAnd how I hurt you\nCan you make it back from the aftermath?\nAnd how I left you\nNobody wants to be alone\nWith the fear of letting go\nIf you could hear me say it's gonna be ok\nWould you be ok?\n\nAnd though I've gone away\nI still see what you're going through\nIt kills me everyday\nTo know I killed what meant most to you\nSo when you pass my grave\nLeave a rose for what might have been\nBut know that it's ok\nYou shed your fears and find love again\n\nFor better or worse, you're the one I never thought I'd hurt\nBut looking back on these dreams\nNothing is what it seems\nKnow that you wake up to better things\n\nAnd though I've gone away\nI still see what you're going through\nIt kills me everyday to know I killed what meant most to you\nSo when you pass my grave\nLeave a rose for what might have been\nBut know that it's ok to shed your fears and find love again\n\nI hear you say\nI don't know how and I don't know why\nBut there ...",bfd04a73e9cffdf0e282c92219a86ea1
4,7624653ca8522ba93470843c74961b7d,"You showed him all the best of you,\nBut I'm afraid your best wasn't good enough\nAnd know he never wanted you at least not the way\nYou wanted yourself to be loved\nAnd you feel like you were a mistake\nHe's not worth all those tears that won't go away\nI wish you could see that\nStill you try to impress hi

In [9]:
lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11414 entries, 0 to 11413
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lyricId   11414 non-null  object
 1   text      11414 non-null  object
 2   track_id  11414 non-null  object
dtypes: object(3)
memory usage: 267.6+ KB


In [10]:
print('Количество строк дубликатов в lyrics -', lyrics.duplicated('track_id').sum())

Количество строк дубликатов в lyrics - 1137


Датасет содержит дубликаты.

## meta

In [11]:
meta.head()

,track_id,dttm,title,language,isrc,genres,duration
0,c3b9d6a354ca008aa4518329aaa21380,1.639688e+12,Happy New Year,EN,RUB422103970,[DANCE],161120.0
1,c57e3d13bbbf5322584a7e92e6f1f7ff,1.637762e+12,Bad Habits,EN,QZN882178276,[ELECTRONICS],362260.0
2,955f2aafe8717908c140bf122ba4172d,1.637768e+12,Por Esa Loca Vanidad,None,QZNJZ2122549,"[FOLK, LATINFOLK]",260000.0
3,fae5a077c9956045955dde02143bd8ff,1.637768e+12,Mil Lagrimas,None,QZNJZ2166033,"[FOLK, LATINFOLK]",190000.0
4,6bede082154d34fc18d9a6744bc95bf5,1.637768e+12,Sexo Humo y Alcohol,None,QZNJZ2122551,"[FOLK, LATINFOLK]",203000.0


In [12]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71769 entries, 0 to 71768
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   track_id  71768 non-null  object 
 1   dttm      71768 non-null  float64
 2   title     71768 non-null  object 
 3   language  21969 non-null  object 
 4   isrc      71455 non-null  object 
 5   genres    71768 non-null  object 
 6   duration  71768 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.8+ MB


In [13]:
print('Количество строк дубликатов в meta -', meta.duplicated('track_id').sum())

Количество строк дубликатов в meta - 0


 - В данных есть пропуски
 - Формат даты добавления определен в секундах
 - Длительность трека содержит большие значения
 - Колонка genres содержит значения в виде списка

## Вывод по этапу "Знакомство с данными"

1. В датасете **covers** не для всех треков присутсвует идентификатор исходного трека.
   Данное условие было озвучено организаторами хакатона.
2. Для датасета **lyrics** обнаружены дубликаты значений в колонке "track_id".
3. Для дадасета **meta**:

    - В данных есть пропуски
    - Колонка "dttm" содержит дату в не интерпритируемом виде
    - Колонка "duration" содержит большие значения
    - Колонка "genres" содержит значения в виде списка



4. Размер всех датасетов разный, а значит не для всех треков будет полная информация.

# Предобработка данных

## Дубликаты в lyrics

Датасет lyrics содержит три колонки: lyric_id, text, track_id.

1. **lyricsId**

    Дубликаты в колонке lyrcisId имеют право быть, так как одна песня может исполняться разными исполнителями и в 
    разной оранжировке.


2. **text**

    Дубликаты текста так же могут присутствовать, как в случае выше.

3. **track_id**

    По сколько track_id - уникальный идентификатор трека, то каждому треку должна соответствовать только
    одна строка датасета. Причина появления таких дубликатов может быть в ошибке выгрузки или ошибке при заполнении 
    информации для датасета.

    Рассмотрим подробнее дубликаты и посмотрим на сколько схожи текста песен для дубликатов уникальных идентификаторов 
    трека. В качестве инструмента будем использовать меру близости Жаккара и n-граммы.

Измени название колонки "lyric_id" на "lyric_id" для удобства использования

In [14]:
lyrics.rename(columns={'lyricId': 'lyric_id'}, inplace=True)

In [15]:
def get_df_for_duplicated_track_id_in_lyrics() -> pd.core.frame.DataFrame:
    
    """Функция вовзращает датафрейм для дубликатов track_id в lyrics.
    Датасет содержит количество текстов для каждого идентификатора,
    список индексов похожести между текстами."""
    
    df = pd.DataFrame(columns=['track_id', 'sym_index', 'num_text'])
    lst_with_track_id = []
    lst_with_index_sym = []
    lst_with_number_text = []

    def jaccard(x: set, y: set):
        shared = x.intersection(y)
        return len(shared) / len(x.union(y))

    def text_to_ngrams(text, N=3):
        return [text[i:i+N] for i in range(len(text)-N+1)]


    duplicated_track_id = lyrics[lyrics.duplicated('track_id')]['track_id'].unique().tolist()
    for track_id in duplicated_track_id:
        df_for_track_id_in_lyrics = lyrics.query('track_id == @track_id')
        df_for_track_id_in_lyrics['text'] = df_for_track_id_in_lyrics['text'].str.lower()
        df_for_track_id_in_lyrics['text'] = df_for_track_id_in_lyrics['text'].apply(lambda x:  re.sub(r"[^\w\s\d]|_", " ", x))
        df_for_track_id_in_lyrics['text'] = df_for_track_id_in_lyrics['text'].apply(lambda x:  re.sub(r'\n', " ", x))
        df_for_track_id_in_lyrics['text'] = df_for_track_id_in_lyrics['text'].apply(text_to_ngrams)
        lst_wth_text = df_for_track_id_in_lyrics['text'].to_list()
        lst_wth_sym_value = []
        number_of_texts = len(lst_wth_text) 
        for i in range(len(lst_wth_text)):
            for j in range(i + 1, len(lst_wth_text)):
                text_first = set(lst_wth_text[i])
                text_second = set(lst_wth_text[j])
                similarity_index = jaccard(text_first, text_second)
                lst_wth_sym_value.append(similarity_index)

        lst_with_track_id.append(track_id)
        lst_with_index_sym.append(lst_wth_sym_value)
        lst_with_number_text.append(number_of_texts) 
    df['track_id'] = lst_with_track_id
    df['sym_index'] = lst_with_index_sym
    df['num_text'] = lst_with_number_text
    return df

In [16]:
df_for_duplicated_track_id_in_lyrics = get_df_for_duplicated_track_id_in_lyrics()
df_for_duplicated_track_id_in_lyrics['index_mean'] = df_for_duplicated_track_id_in_lyrics['sym_index'].progress_apply(np.mean)
df_for_duplicated_track_id_in_lyrics.head(10)

  0%|          | 0/637 [00:00<?, ?it/s]

,track_id,sym_index,num_text,index_mean
0,ea09c90790b997b6e384d75d424b3ffe,"[0.6012269938650306, 0.46445497630331756, 0.5764705882352941, 0.63125, 0.6908212560386473, 0.9192546583850931, 0.89937106918239, 0.6839622641509434, 0.6509433962264151, 0.9192546583850931]",5,0.703701
1,dec84e10ae3ecc5c363f495724799322,"[0.9774011299435028, 0.9441340782122905, 0.9630681818181818, 0.9441340782122905, 0.9495798319327731, 0.9519774011299436, 0.9495798319327731, 0.9352112676056338, 1.0, 0.9352112676056338]",5,0.955030
2,db69f2a265624a834d6e2a1238f6f225,"[0.6193353474320241, 0.6121212121212121, 0.6330275229357798, 0.4163934426229508, 0.9518072289156626, 0.9637462235649547, 0.6309523809523809, 0.93993993993994, 0.6484848484848484, 0.6201780415430267]",5,0.703599
3,a367959976d3ca166beb46bdf66eefd0,"[1.0, 0.939922480620155, 0.939922480620155]",3,0.959948
4,9837ed3cc108b811b0b8225680784d43,[0.9578947368421052],2,0.957895
5,4500cc1d632ca30c29eb64fd4c41cef9,[0.8844765342960289],2,0.884477
6,f251ad6a4f1c1c940777fe5b61307d0a,[0.968],2,0.968000
7,c0c707cc4dc47ca3c4326d259a083756,[0.8952879581151832],2,0.895288
8,371380c62849548617f1ef5debef2a5c,[0.8826714801444043],2,0.882671
9,bfa95662bfdb59ebc842cb19bfc32f67,[0.9511343804537522],2,0.951134


У большинства дубликатов высокий показатели похожести для текстов.

Но так же есть дубликаты, тексты песен для которого имеют сильные различия. Рассмотрим их.

In [17]:
df_for_duplicated_track_id_in_lyrics.query('index_mean <= 0.7')

,track_id,sym_index,num_text,index_mean
24,7b819ca5db44e465b28b1bdd34792add,[0.484375],2,0.484375
35,b626ec4ba92f2bb092ff5496ad7e894b,"[0.9786096256684492, 0.9786096256684492, 0.9786096256684492, 0.3279569892473118, 0.33153638814016173, 0.33153638814016173, 1.0, 1.0, 0.3297587131367292, 0.3333333333333333, 0.3333333333333333, 1.0, 0.3297587131367292, 0.3333333333333333, 0.3333333333333333, 0.3297587131367292, 0.3333333333333333, 0.3333333333333333, 0.953125, 0.953125, 1.0]",7,0.610590
46,4ca2ed2cd7a311621e8a2648b1c75f6b,"[0.36585365853658536, 1.0, 0.6521739130434783, 0.36585365853658536, 0.38461538461538464, 0.6521739130434783]",4,0.570112
61,0f91fd9e4a14220bbe9683cbea37c04a,"[0.8641425389755011, 0.12385919165580182, 0.9578454332552693, 0.8641425389755011, 1.0, 0.13175230566534915, 0.8552338530066815, 1.0, 0.8641425389755011, 0.12894736842105264, 0.13175230566534915, 0.12385919165580182, 0.8552338530066815, 0.9578454332552693, 0.8641425389755011]",6,0.648193
70,b8602690a53fae82847becb1134c07fe,"[0.13175230566534915, 0.8552338530066815, 1.0, 0.8641425389755011, 0.8641425389755011, 0.12894736842105264, 0.13175230566534915, 0.12385919165580182, 0.12385919165580182, 0.8552338530066815, 0.9578454332552693, 0.9578454332552693, 0.8641425389755011, 0.8641425389755011, 1.0]",6,0.648193
80,f7ab70f5c354d1fd5c3fdb41da95d716,"[0.5705645161290323, 0.0, 0.8732673267326733, 0.0, 0.532803180914513, 0.0]",4,0.329439
95,cbd051407e37447586f47548a855d3a1,"[1.0, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.9257425742574258, 0.9257425742574258]",6,0.695713
96,3bc9114a776eb35c71f0a0821e6d6b97,"[1.0, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.9257425742574258, 0.9257425742574258]",6,0.695713
97,8bda124571bf6555c4c9432182de9a83,"[1.0, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.9257425742574258, 0.9257425742574258]",6,0.695713
98,f4cb440c53078ede69df473e3974c30f,"[1.0, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 0.5089058524173028, 0.5089058524173028, 0.5102564102564102, 1.0, 0.9257425742574258, 0.9257425742574258]",6,0.695713


In [18]:
lyrics[lyrics['track_id'] == 'bfc09d4058ee2478268b51f2a8fd4bda']

,lyric_id,text,track_id
4711,fd28d472292ab7638c11f6db6293ac87,"Since I was born they couldn't hold me down\nAnother misfit kid, another burned-out town\nNever played by the rules, I never really cared\nMy nasty reputation takes me everywhere\n\nI look and see it's not only me\nSo many others have stood where I stand\nWe are the young so raise your hands\n\nThey call us problem child\nWe spend our lives on trial\nWe walk an endless mile\nWe are the youth gone wild\nWe stand and we won't fall\nWe're the one and one for all\nThe writing's on the wall\nWe are the youth gone wild\n\nBoss screamin' in my ear about who I'm supposed to be\nGetcha a 3-piece Wall Street smile and son you'll look just like me\nI said ""Hey man, there's something that you oughta know.\nI tell ya Park Avenue leads to Skid Row.""\n\nI look and see it's not only me\nWe're standing tall ain't never a doubt\nWe are the young, so shout it out\n\nThey call us problem child\nWe spend our lives on trial\nWe walk an endless mile\nWe are the youth gone wild\nWe stand and we won't fall...",bfc09d4058ee2478268b51f2a8fd4bda
7968,4b6d60f6d47e17dadecc67580f2fa3f3,"Every breath you take\nI watch you slip away\nYou're slowly killing yourself\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\nI won't give in\n\nIf my heart could sing, would you stay?\nWould you stay and listen?\nWould you stay and listen?\n\nIf my soul was torn, would you help?\nWould you try and fix me?\nWould you help un-break me?\n\nYour smile, it eats me alive\nAnd I can't turn away any longer\n\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\nI won't give in\n\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\n\nI won't give in\nI'm a ghost of what's left of me\nBegging you to hear me\n\nCan you even feel me?\nI can't let you go, save me please\n\nWill you stay here with me?\nWill you ever leave me?\nYour smile it eats me alive\nI can't turn away any longer\n\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\nI won't give in\n\nEvery breath you take\nI watch yo...",bfc09d4058ee2478268b51f2a8fd4bda


In [19]:
lyrics[lyrics['track_id'] == '63947ae3848501f86fba6a7d3bb9f598']

,lyric_id,text,track_id
4261,7f3bf431e1648c8c3bc82e620e646796,"Oh, I guess it would be nice\nIf I could touch your body\nI know not everybody has got a body like me\nBut I got to think twice\nBefore I give my heart away\nAnd I know all the games you play 'cause I play them too\n\nOh, but I need some time off from that emotion\nTime to pick my heart up off the floor\nWhen that love comes down without devotion\nWell, it takes a strong man, baby\nBut I'm showing you the door\n\nI gotta have faith\nGotta have faith\nGotta have faith\nGotta have faith\n\nBaby, I know you're asking me to stay\nSay please, please, please, don't go away\n'Cause you're giving me the blues\nMaybe you might mean all the words you say\nCan't help but think of yesterday\nAnd another who tied me down to the loverboy blues\n\nBefore this river becomes an ocean\nBefore you pick my heart up off the floor\nWhen our love comes down without devotion\nWell, it takes a strong man, baby\nBut I'm showing you that door\n\nI gotta have faith\nGotta have faith\nGotta have faith\nGotta h...",63947ae3848501f86fba6a7d3bb9f598
4715,f9705d5f8a75893d0484e3376452df90,"Am I freak in the darkness, or am I misfit\nYou speak of opinions\nDo sink in so deep\nBut it's alright\nYou're just an illusion, confused by your narrow mind\nReality is up ahead in the distance\nBut that lack of persistence has left you behind\nNow you're reaching for your sanity\n'Cause you afraid of me\nSo don't fuck with me\nYou wanna ask me a question\nWell, I gotta question\nHow much longer can I tolerate this shit\nEgos trip when you're livin' on the flip side\nDrop out of a uterous and god damn\nI see you pointin' your finger\nYou stereotype me 'cause you don't like me\nWell you don't even know me\nYeah punk, you don't know me\n\nStereotype me\n'Cause you don't like me",63947ae3848501f86fba6a7d3bb9f598


In [20]:
lyrics[lyrics['track_id'] == 'bfc09d4058ee2478268b51f2a8fd4bda']

,lyric_id,text,track_id
4711,fd28d472292ab7638c11f6db6293ac87,"Since I was born they couldn't hold me down\nAnother misfit kid, another burned-out town\nNever played by the rules, I never really cared\nMy nasty reputation takes me everywhere\n\nI look and see it's not only me\nSo many others have stood where I stand\nWe are the young so raise your hands\n\nThey call us problem child\nWe spend our lives on trial\nWe walk an endless mile\nWe are the youth gone wild\nWe stand and we won't fall\nWe're the one and one for all\nThe writing's on the wall\nWe are the youth gone wild\n\nBoss screamin' in my ear about who I'm supposed to be\nGetcha a 3-piece Wall Street smile and son you'll look just like me\nI said ""Hey man, there's something that you oughta know.\nI tell ya Park Avenue leads to Skid Row.""\n\nI look and see it's not only me\nWe're standing tall ain't never a doubt\nWe are the young, so shout it out\n\nThey call us problem child\nWe spend our lives on trial\nWe walk an endless mile\nWe are the youth gone wild\nWe stand and we won't fall...",bfc09d4058ee2478268b51f2a8fd4bda
7968,4b6d60f6d47e17dadecc67580f2fa3f3,"Every breath you take\nI watch you slip away\nYou're slowly killing yourself\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\nI won't give in\n\nIf my heart could sing, would you stay?\nWould you stay and listen?\nWould you stay and listen?\n\nIf my soul was torn, would you help?\nWould you try and fix me?\nWould you help un-break me?\n\nYour smile, it eats me alive\nAnd I can't turn away any longer\n\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\nI won't give in\n\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\n\nI won't give in\nI'm a ghost of what's left of me\nBegging you to hear me\n\nCan you even feel me?\nI can't let you go, save me please\n\nWill you stay here with me?\nWill you ever leave me?\nYour smile it eats me alive\nI can't turn away any longer\n\nEvery breath you take\nI watch you slip away\nYou're slowly killing yourself\nI won't give in\n\nEvery breath you take\nI watch yo...",bfc09d4058ee2478268b51f2a8fd4bda


Как видно из примеров выше, тексты для одного итого же трека могут различаться как по длине самого текста так и содержать отличающийся набор слов.

Кроме того, у нас нет отметки о том, какой текст является исходным.

Возможно трек подгружают из разных источников. Добавим информацию из датасетов covers, meta и оценим полученную информацию.

In [21]:
dct_wth_target = dict(zip(covers['track_id'], covers['track_remake_type']))
dct_wth_original_track_id = dict(zip(covers['track_id'], covers['original_track_id']))

df_for_duplicated_track_id_in_lyrics = df_for_duplicated_track_id_in_lyrics.merge(meta, on='track_id', how='left')
df_for_duplicated_track_id_in_lyrics['track_remake_type'] = df_for_duplicated_track_id_in_lyrics['track_id'].progress_apply(lambda x: dct_wth_target.get(x, 'unknown'))
df_for_duplicated_track_id_in_lyrics['original_track_id'] = df_for_duplicated_track_id_in_lyrics['track_id'].progress_apply(lambda x: dct_wth_target.get(x, 'unknown'))

  0%|          | 0/637 [00:00<?, ?it/s]

  0%|          | 0/637 [00:00<?, ?it/s]

In [22]:
df_for_duplicated_track_id_in_lyrics['track_remake_type'].value_counts()

track_remake_type
COVER       297
ORIGINAL    280
unknown      60
Name: count, dtype: int64

60 треков в lyrics без разметки целевого признака, но с наличием информации в meta.

In [23]:
df_for_duplicated_track_id_in_lyrics['original_track_id'].value_counts().head()

original_track_id
COVER       297
ORIGINAL    280
unknown      60
Name: count, dtype: int64

Для некоторых 60 треков так же отсутствуют сведения об оригинальном треке.

Проверим, не совпадают ли эти треки по названиям. Для этого сравним треки из двух групп по "track_id".

In [24]:
df_for_duplicated_track_id_in_lyrics.query('track_remake_type == "unknown"')['track_id'].unique().tolist() == \
df_for_duplicated_track_id_in_lyrics.query('original_track_id == "unknown"')['track_id'].unique().tolist()

True

Те же 60 треков без разметки о типе трека не имеют данных об оригинальном треке.

Кроме того,  мы будет строить будущую модель бинарной классификации, признак "track_remake_type" будет являться целевым. Учитывая, что треков немного, рациональным решением будет удалить их. 

In [25]:
lyrics =\
lyrics[~(lyrics['track_id'].isin(df_for_duplicated_track_id_in_lyrics.query('track_remake_type == "unknown"')['track_id'].unique().tolist()))]

df_for_duplicated_track_id_in_lyrics = df_for_duplicated_track_id_in_lyrics.query('track_remake_type != "unknown"')

Теперь рассмотрим треки-оригиналы с дубликатами текстов. У нас уже есть примерные сравнения дубликатов текста для каждого трека. Если текст подгружали из разных источников, то скорее всего наиболее правильный будет содержать большее количество символов, в случае если текста идентичны, то оставим один из них.

In [26]:
lyrics['num_sym_in_text'] = lyrics['text'].progress_apply(lambda x: len(x))

  0%|          | 0/11191 [00:00<?, ?it/s]

In [27]:
df_for_duplicated_track_id_in_lyrics.query('(track_remake_type == "ORIGINAL")')

,track_id,sym_index,num_text,index_mean,dttm,title,language,isrc,genres,duration,track_remake_type,original_track_id
3,a367959976d3ca166beb46bdf66eefd0,"[1.0, 0.939922480620155, 0.939922480620155]",3,0.959948,1.466104e+12,Heathens,EN,USAT21601930,"[FILMS, SOUNDTRACK]",195910.0,ORIGINAL,ORIGINAL
5,4500cc1d632ca30c29eb64fd4c41cef9,[0.8844765342960289],2,0.884477,1.642799e+12,Where's My Love,EN,CAN111600624,[INDIE],239880.0,ORIGINAL,ORIGINAL
7,c0c707cc4dc47ca3c4326d259a083756,[0.8952879581151832],2,0.895288,1.678741e+12,Italia,None,SMRUS0074406,"[POP, RUSPOP]",164320.0,ORIGINAL,ORIGINAL
8,371380c62849548617f1ef5debef2a5c,[0.8826714801444043],2,0.882671,1.500930e+12,Believer,EN,USUM71700626,"[ROCK, ALLROCK]",204330.0,ORIGINAL,ORIGINAL
10,dab794931ab35c33a9945be64e276500,[0.9014598540145985],2,0.901460,1.642712e+12,Where's My Love,EN,CAN111700009,[INDIE],202100.0,ORIGINAL,ORIGINAL
12,f5b1b6321334fd64cd81d04b024e52a0,"[1.0, 0.7632933104631218, 0.6759581881533101, 0.7632933104631218, 0.6759581881533101, 0.7967479674796748]",4,0.779208,1.263244e+12,The Way I Are,EN,USUM70734367,"[POP, RAP]",199560.0,ORIGINAL,ORIGINAL
16,af5c11a1591bcbf7102e247b756ddf1b,"[0.8969298245614035, 0.918552036199095, 0.8387096774193549, 0.8866666666666667, 0.9048672566371682, 0.8183856502242153, 0.8617710583153347, 0.825287356321839, 0.9160997732426304, 0.815068493150685]",5,0.868234,1.437080e+12,Sugar,EN,DEA621501057,[DANCE],219040.0,ORIGINAL,ORIGINAL
18,995665640dc319973d3173a74a03860c,"[0.7632933104631218, 0.7899305555555556, 0.7632933104631218, 0.9021956087824351, 1.0, 0.9021956087824351]",4,0.853485,1.258405e+12,The Way I Are,EN,USUM70722806,"[FOREIGNRAP, RAP]",179660.0,ORIGINAL,ORIGINAL
19,8988e304388ebac045986bfcd29fbaaa,"[1.0, 0.9714285714285714, 0.9876543209876543, 0.9958677685950413, 0.9958677685950413, 0.8968253968253969, 0.9958677685950413, 0.9714285714285714, 0.9876543209876543, 0.9958677685950413, 0.9958677685950413, 0.8968253968253969, 0.9958677685950413, 0.967479674796748, 0.9755102040816327, 0.9755102040816327, 0.8932806324110671, 0.9755102040816327, 0.9917695473251029, 0.9917695473251029, 0.8932806324110671, 0.9917695473251029, 1.0, 0.9007936507936508, 1.0, 0.9007936507936508, 1.0, 0.9007936507936508]",8,0.965903,1.330546e+12,What Is Love,EN,DEA410500401,[POP],268700.0,ORIGINAL,ORIGINAL
27,127bed698777289998943260b43b3e9d,[0.9076433121019108],2,0.907643,1.397156e+12,Трудный возраст,RU,RUA320500001,"[POP, RUSPOP]",224000.0,ORIGINAL,ORIGINAL


In [28]:
def get_lyric_id_wth_min_len_text(df):
    lst_wth_lirics_id = []
    lst_for_remove = []
    ids = df['track_id'].unique().tolist()
    for i in tqdm(ids):
        df_for_i = lyrics.query('track_id == @i')
        dct_lyrics_id = dict(zip(df_for_i['lyric_id'].tolist(), df_for_i['num_sym_in_text'].tolist()))
        key = max(dct_lyrics_id, key=dct_lyrics_id.get)
        dct_lyrics_id.pop(key)
        lst_for_remove.append([i, list(dct_lyrics_id.keys())])
    return lst_for_remove

In [29]:
lyrics['will_del'] = 0

In [30]:
lst_with_lyr_id_min_len_text_for_original_track =\
get_lyric_id_wth_min_len_text(df_for_duplicated_track_id_in_lyrics.query('track_remake_type == "ORIGINAL"'))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 399.43it/s]


In [31]:
def mark_dupl(df, lst):
    for c in tqdm(lst):
        for lyr_id in c[1]:
            df.loc[(df['track_id'] == c[0]) & (df['lyric_id'] == lyr_id), 'will_del'] = 1
    return df

In [32]:
lyrics = mark_dupl(lyrics, lst_with_lyr_id_min_len_text_for_original_track)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280/280 [00:01<00:00, 214.48it/s]


“Ковер” - это термин, используемый в музыке для описания исполнения или перепевки существующей песни другим артистом или группой. Обычно это делается с целью представить новую интерпретацию песни или просто для развлечения.

Степень соответствия текста ковра оригиналу может значительно варьироваться. Некоторые каверы сохраняют оригинальный текст и мелодию песни, в то время как другие могут изменять их, чтобы соответствовать стилю и голосу исполнителя. Также бывают случаи, когда ковер содержит совершенно новый текст, основанный на оригинальной мелодии.

В случае с коверами поступи аналогичным образом и сотавим наиболее полный (по количеству символов текст) текст.

In [33]:
df_for_duplicated_track_id_in_lyrics.query('(track_remake_type == "COVER")')

,track_id,sym_index,num_text,index_mean,dttm,title,language,isrc,genres,duration,track_remake_type,original_track_id
0,ea09c90790b997b6e384d75d424b3ffe,"[0.6012269938650306, 0.46445497630331756, 0.5764705882352941, 0.63125, 0.6908212560386473, 0.9192546583850931, 0.89937106918239, 0.6839622641509434, 0.6509433962264151, 0.9192546583850931]",5,0.703701,1.380139e+12,Hound Dog,EN,DELM40900205,"[ALLROCK, RNR]",134670.0,COVER,COVER
1,dec84e10ae3ecc5c363f495724799322,"[0.9774011299435028, 0.9441340782122905, 0.9630681818181818, 0.9441340782122905, 0.9495798319327731, 0.9519774011299436, 0.9495798319327731, 0.9352112676056338, 1.0, 0.9352112676056338]",5,0.955030,1.467900e+12,Let Me Hold You (Turn Me On),EN,NLZ541600032,[DANCE],162420.0,COVER,COVER
4,9837ed3cc108b811b0b8225680784d43,[0.9578947368421052],2,0.957895,1.537532e+12,Smile,EN,USQX90900707,[POP],181930.0,COVER,COVER
6,f251ad6a4f1c1c940777fe5b61307d0a,[0.968],2,0.968000,1.484124e+12,Human,EN,TCACW1759000,[POP],197300.0,COVER,COVER
9,bfa95662bfdb59ebc842cb19bfc32f67,[0.9511343804537522],2,0.951134,1.522357e+12,Go Flex,EN,USAT21801066,"[FOREIGNBARD, BARD]",194820.0,COVER,COVER
11,522be89f006a63641278bb0055764f42,[0.9563106796116505],2,0.956311,1.552597e+12,Can't Help Falling In Love,EN,USNLR1800566,[SOUNDTRACK],201930.0,COVER,COVER
13,dc0c11952bfc275612cbc3986021684f,"[1.0, 0.6857142857142857, 0.7989949748743719, 0.6857142857142857, 0.7989949748743719, 0.8594594594594595]",4,0.804813,1.257455e+12,Cocaine,EN,NLF057790024,"[ROCK, BLUES, ALLROCK]",218100.0,COVER,COVER
15,2c482187de3d9a7957b5c28cac7e8e3a,[0.9760479041916168],2,0.976048,1.656774e+12,La Escuelita,None,QZNJV2295573,"[FOLK, LATINFOLK]",128000.0,COVER,COVER
17,c5da3ea241a8ac20e402d2279a26a20a,"[1.0, 1.0, 1.0]",3,1.000000,1.680296e+12,Livin' my life,None,None,[DANCE],396140.0,COVER,COVER
20,ed280fd343a016d46ba33e7338f18fcf,[0.9841269841269841],2,0.984127,1.608239e+12,Cry for You,EN,DEMA61902450,[DANCE],138200.0,COVER,COVER


In [34]:
lst_with_lyr_id_min_len_text_for_cover_track =\
get_lyric_id_wth_min_len_text(df_for_duplicated_track_id_in_lyrics.query('track_remake_type == "COVER"'))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297/297 [00:00<00:00, 371.99it/s]


In [35]:
lyrics = mark_dupl(lyrics, lst_with_lyr_id_min_len_text_for_cover_track)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297/297 [00:01<00:00, 222.56it/s]


In [36]:
lyrics = lyrics[lyrics['will_del'] != 1]

In [37]:
lyrics.drop(['will_del'], axis=1, inplace=True)

In [38]:
print('Количество дубликатов после обработки -', lyrics['track_id'].duplicated().sum())

Количество дубликатов после обработки - 0


In [39]:
del df_for_duplicated_track_id_in_lyrics, get_df_for_duplicated_track_id_in_lyrics, dct_wth_target, dct_wth_original_track_id

In [40]:
del lst_with_lyr_id_min_len_text_for_original_track, get_lyric_id_wth_min_len_text, lst_with_lyr_id_min_len_text_for_cover_track

In [41]:
del mark_dupl

## meta

Прежде чем обрабатывать длительность и дату, проверим аномальные значения(0 или отрицательные значения)

In [42]:
meta['dttm'].describe()

count    7.176800e+04
mean     1.584287e+12
std      9.113923e+10
min      1.249926e+12
25%      1.570141e+12
50%      1.620405e+12
75%      1.637050e+12
max      1.697663e+12
Name: dttm, dtype: float64

С датой все впорядке на первый взгляд.

In [43]:
meta['duration'].describe()

count    7.176800e+04
mean     2.049187e+05
std      8.559854e+04
min      0.000000e+00
25%      1.613700e+05
50%      1.993950e+05
75%      2.390700e+05
max      5.487300e+06
Name: duration, dtype: float64

А для длительности трека присутствуют  нулевые значения. Если их немного, то просто удалим их.

In [44]:
print('Количество нулевых значений в датасете -', len(meta[meta['duration'] == 0]))

Количество нулевых значений в датасете - 369


In [45]:
meta = meta[meta['duration'] != 0]

Теперь скорректируем формат даты и длительность трека.

Длительность трека скорее всего выражена в миллисекундах, сконвертируем ее в минуты.
Аналогично поступи и для даты.

In [46]:
meta['dttm'] = pd.to_datetime(meta['dttm'], unit='ms')
meta['duration'] = meta['duration'].progress_apply(lambda x: x / 60000)

  0%|          | 0/71400 [00:00<?, ?it/s]

Посмотрим на полученые значения длительности трека

In [47]:
meta['duration'].describe()

count    71399.000000
mean         3.432962
std          1.408983
min          0.042833
25%          2.700500
50%          3.329167
75%          3.989000
max         91.455000
Name: duration, dtype: float64

In [48]:
meta[meta['duration'] < 0.5].head(15)

,track_id,dttm,title,language,isrc,genres,duration
540,0581250d585e24b105aa476318b399b0,2021-12-13 21:00:00,"Feeding a Hungry Luma (From ""Super Mario Galaxy"")",None,AUXN22212741,[ELECTRONICS],0.155833
547,d0e7202abb848a1a62af2dae9e0bb37d,2021-12-13 21:00:00,"Got a Star (From ""Super Mario Galaxy"")",None,AUXN22212748,[ELECTRONICS],0.101500
548,8c8a61a720ca9ddac6439799606eedb1,2021-12-13 21:00:00,"Grand Star, Get (From ""Super Mario Galaxy"")",None,AUXN22212749,[ELECTRONICS],0.214000
550,60309277d6c1973615b6f436323bf74d,2021-12-13 21:00:00,"Hungry Luma (From ""Super Mario Galaxy"")",None,AUXN22212751,[ELECTRONICS],0.123833
562,48b1ecd07ae91b484ee5d2438cc1fa1f,2021-12-13 21:00:00,"Star Babies (From ""Super Mario Galaxy"")",None,AUXN22212763,[ELECTRONICS],0.494333
571,02d8f973b43650440acfbb02b56b230d,2021-12-20 21:00:00,"Big Rock Finish (From ""Rhythm Heaven"")",None,AUXN22213018,[ELECTRONICS],0.217833
583,06267d909164599bb37719237bd6544a,2021-12-20 21:00:00,"I Gotta Sing (From ""Rhythm Heaven"")",None,AUXN22213030,[ELECTRONICS],0.042833
591,f745fb855bc31b847cf987c1b7a64fe8,2021-12-20 21:00:00,"PIC (From ""Rhythm Heaven"")",None,AUXN22213038,[ELECTRONICS],0.294500
624,726fcca2b00e4e1bf9926cffcb5d78c0,2021-12-06 21:00:00,"Cloudy Climb (From ""Paper Mario"")",None,AUXN22212905,[ELECTRONICS],0.448667
635,b24b68f75573ebd44288fe857238891b,2021-12-06 21:00:00,"E-Mail from X (From ""Paper Mario, The Thousand-Year Door"")",None,AUXN22212916,[ELECTRONICS],0.312167


Некоторые треки длятся менее 0.05. По названию и длительности похоже, что это не композиции, а что-то вроде сэмпла озвучивания. Например, из какой-нибудь игры или ригтон для звонка. В дальнейшем выделим для таких треков отдельную группу.

In [49]:
meta['dttm'].describe()

count                            71399
mean     2020-03-16 09:37:18.142593280
min                2009-08-10 17:32:06
25%                2019-10-09 21:00:00
50%                2021-05-07 22:16:50
75%                2021-11-15 21:00:00
max                2023-10-18 21:00:00
Name: dttm, dtype: object

С датами все впорядке.

**Пропуски в language**

Датасет содержит большое клоичество пропусков в колонке. Восстанавливать язык по названию трека будет не корректным шагом, так как название не всегда является отражением языка исполнения.

Более точным шагом будет определение языка по тексту песни,
но текстов песен гораздо меньше, пропусков в meta.

На данном этапе оставим пропуски без заполнения. А заполним их по тексту песни после того, как соберем единный датасет.

**Пропуски в genres**

В колонке жанр присутстсвуют нулевые значения.

In [50]:
meta['genres'] = meta['genres'].progress_apply(lambda x: ' '.join(i for i in x) if x is not None else x)

  0%|          | 0/71400 [00:00<?, ?it/s]

In [51]:
print('Количество треков с неуказаннм жанром -', len(meta[meta['genres'] == ""]))

Количество треков с неуказаннм жанром - 2352


In [52]:
meta[meta['genres'] == ""]

,track_id,dttm,title,language,isrc,genres,duration
275,3fcb3d72aaf4f97cec177ae3ec9e01b7,2021-11-29 13:48:17,Carol Of The Bells,None,QZNJZ2196497,,1.511000
276,0ab5de9e23590298e4e3f0127ee53a5d,2021-11-29 13:48:17,Blanca Navidad,None,QZNJZ2196500,,3.010500
277,01886afe7dd91a24c86273ab17c62d21,2021-11-29 13:48:17,Rodolfo el Reno,None,QZNJZ2196498,,2.195500
278,67fe0561d7fe3d1f82ff200315098988,2021-11-29 13:48:17,A la Nanita Nana,None,QZNJZ2196499,,2.592500
279,c3c396410697a76825830fcd12d866b5,2021-11-29 13:48:17,Let It Snow! Let It Snow! Let It Snow!,None,QZNJZ2196501,,2.558333
...,...,...,...,...,...,...,...
71532,6a80cf1d895ac7c356185c726fb97dde,2019-05-17 17:51:19,Buwan,None,PLS921748465,,2.889667
71533,b001fb3f5d9c43946a07c0e5cc393f70,2019-05-17 17:51:19,Nothing's Gonna Stop Us Now,None,PLS921748932,,3.243333
71534,92010023163a575570ed02ffb01cf26e,2019-05-17 17:51:19,Roads,None,PLS921752755,,4.277833
71535,4fd058e3ed46bcfcf30ca156515feec6,2019-05-17 17:51:19,Eye of the Tiger,None,PLS921748828,,2.503833


In [53]:
meta[meta['title'] == 'Castle of Glass']

,track_id,dttm,title,language,isrc,genres,duration
69151,a429d4e3e9cbf892d07968f55e9e931b,2019-02-20 03:33:23,Castle of Glass,None,PLS921753182,,3.626333
70034,8bfd55d0c7f9990e6b181ba96dc325b6,2019-03-19 07:41:15,Castle of Glass,None,PLS921753285,CLASSICAL CLASSICALMUSIC,4.790000
71361,6dd63e1b241053563adea30b82850677,2019-05-14 17:45:36,Castle of Glass,None,PLS921753327,,3.719000
71716,a12cf3d5db7975f54354a4b304bad783,2019-06-07 05:51:11,Castle of Glass,None,PLS921753230,,3.652667


На данный момент нет корректного способа указать жанр для этих композиций. Заменим отсутстсвующие значения на "unknown".

In [54]:
meta.loc[meta['genres'] == "", 'genres'] = 'unknown'

**Пропуски в isrc**

Пропуски в клонке могут быть обусловлены:

    1. Композиция очень старая
    2. Композиция записана на местном локальном уровне и не сотрудничает с крупными лейблами
    3. Композиция может быть записана как демо
    4. Композиция выпущена не официально (любительская/любительская-студийная запись)
    5. Ошибка выгрузки или заполнения данных

Прежде чем заполнять пропуски, проверим на на наличие дубликатов для колонки isrc.

In [55]:
print('Количество дубликатов для isrc -', meta['isrc'].duplicated().sum())

Количество дубликатов для isrc - 459


ISRC (International Standard Recording Code) представляет собой уникальный код, присваиваемый каждой конкретной аудиозаписи. Каждый трек или музыкальное произведение должно иметь свой уникальный ISRC. Этот код позволяет отслеживать и идентифицировать конкретную запись или трек в цифровой музыкальной индустрии.

Использование одного и того же ISRC для двух разных треков нарушает стандарты и может вызвать проблемы при управлении авторскими правами, отслеживании продаж, выплате гонораров и т.д. А значит, что каждый трек должен иметь один уникальный ISRC.

In [56]:
meta[meta['isrc'].isin(meta[meta['isrc'].duplicated()]['isrc'].unique().tolist())].sort_values('isrc').head()

,track_id,dttm,title,language,isrc,genres,duration
26955,8aff9d28865aaf1fa44e46b82cba0f63,2021-07-06 15:38:37,Burma,None,ATAJ12100470,FOLK,2.988333
27012,be0233f57dbb59079929ac646b0b9d5c,2021-07-06 16:38:30,Zivot da stane ne sme,None,ATAJ12100470,FOLK,2.519833
66395,425761c88ebc69e89c748eb537671953,2018-04-28 19:08:26,Inni Mer Syngur Vitleysingur,None,ATN261668711,FOLKMETAL METAL,3.950167
51052,196a0973d73122891a426f8334105ce7,2019-03-29 21:00:00,Inní mér syngur vitleysingur,EN,ATN261668711,FOLKMETAL METAL,3.950167
51053,aa4a1d4d44890cbc7aae285842bd3ce2,2019-03-29 21:00:00,Nattfödd,EN,ATN261668712,FOLKMETAL METAL,4.255333


Серди дубликатов по isrc присутствуют записи как с одинаковыми значениями для title, duration и isrc, так и с разынми названиями треков. При этом идентификатор трека у всех разный.

Возможно это могут быть и оригинальные треки и коверы, но идентификатор могли указать на оригиналльный трек.
Так как у нас есть уникальный идентификатор самого трека и колонка для него не содержит дубликатов,
предлагаем оставить "дубликаты" без изменений.

Появляется вопрос в корректности заполнения колонки isrc, и целесообразности ее использования в дальнейшем.

In [57]:
print('Количество пропусков для isrc -', meta['isrc'].isna().sum())

Количество пропусков для isrc - 314


Для восстановления isrcs напишем функцию, которая будет обращатся через API к https://musicbrainz.org.

Нас будут интересовать только те композиции, у которых полностью совпадает название и есть идентификатор.

В дполнение, сделаем признак-метку - был ли isrc изначально. Считаем, что такая информация может пригодится.

In [58]:
meta['is_isrc'] = meta['isrc'].progress_apply(lambda x: 0 if pd.isna(x) else 1)

  0%|          | 0/71400 [00:00<?, ?it/s]

In [59]:
def get_isrc_for_track(track_titles: list) -> pd.core.frame.DataFrame:
    art_name_dq = deque()
    title_dq = deque()
    duration_dq = deque()
    isrc_dq = deque()

    for track_title in tqdm(track_titles):
        params = {"query": track_title,
                  "limit": 50,
                  "fmt": "json"}
        response = requests.get(f"https://musicbrainz.org/ws/2/recording", params=params)
        if response.status_code == 200:
            data = response.json()
            for rec in data['recordings']:
                isrc = rec.get("isrcs", "no")
                founded_title = rec.get("title", "no")
                if (founded_title) and (track_title):
                    if (isrc != "no") and ((founded_title).lower() == (track_title).lower()):
                        isrc = isrc[0]
                        artist_name = rec.get("artist-credit", [{}])[0].get("artist", {}).get("name", "no")
                        duration = rec.get("length", "no")
                        lang = 'no'

                        if not founded_title.isnumeric():
                            art_name_dq.append(artist_name)
                            title_dq.append(founded_title)
                            duration_dq.append(duration)
                            isrc_dq.append(isrc)
        
    df = pd.DataFrame.from_dict({'artist_name': list(art_name_dq),
                                 'title': list(title_dq),
                                 'duration': list(duration_dq),
                                 "isrc": list(isrc_dq)})
    df['duration'] = df['duration'].apply(lambda x: x / 60000 if (type(x) is int) or (type(x) is float) else x)    
    return df

In [60]:
df_for_na_in_isrc = get_isrc_for_track(meta[meta['isrc'].isna()]['title'].unique().tolist())

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297/297 [02:03<00:00,  2.41it/s]


Для идентификации isrc попробуем сравнить продолжительность найденных треков с треками в meta.

Если найдем похожие пары (значение должно быть либо равно, либо максимально близко к полученному ранее), то определим isrc.

Для начала добавим длительность трека из meta. 

Если треков с таким названием несколько, то добавим все временные метки.
Если меток из meta нет, то присвоим найденный isrc треку с таким названием.

In [61]:
df_for_na_in_isrc.drop_duplicates(subset=['isrc'], inplace=True)

In [62]:
df_for_na_in_isrc['dur_from_meta'] = df_for_na_in_isrc['title'].progress_apply(lambda x: meta[meta['isrc'].isna()].query('title == @x')['duration'].tolist())

  0%|          | 0/190 [00:00<?, ?it/s]

In [63]:
print('Уникальные значения для количества временных меток из meta', 
      df_for_na_in_isrc['dur_from_meta'].progress_apply(lambda x: len(x)).unique().tolist())

  0%|          | 0/190 [00:00<?, ?it/s]

Уникальные значения для количества временных меток из meta [1, 2, 0]


In [64]:
def find_close_num(row):
    res = None
    lst_dur = row['dur_from_meta']
    target = row['duration']
    if (len(lst_dur) > 0) and (target != 'no'):
        dct_with_diff = {}
        for i in range(len(lst_dur)):
            diff = abs(float(target) - float(lst_dur[i]))
            dct_with_diff[str(lst_dur[i])] = diff
        res = min(dct_with_diff, key=dct_with_diff.get)
    return res

In [65]:
df_for_na_in_isrc['eq_dur_from_meta'] = df_for_na_in_isrc.progress_apply(find_close_num, axis=1)

  0%|          | 0/190 [00:00<?, ?it/s]

In [66]:
df_for_na_in_isrc.head()

,artist_name,title,duration,isrc,dur_from_meta,eq_dur_from_meta
0,Lady Gaga,Americano,4.1,USUM72212805,[3.763333333333333],3.763333333333333
1,ЧипаЧип,Круги на воде,2.900317,RUA1D1919888,"[3.547, 4.686]",3.547
2,Майя Кристалинская,Круги на воде,4.231783,RUAAX2100580,"[3.547, 4.686]",4.686
3,Майя Кристалинская,Круги на воде,4.266067,FR6V82600821,"[3.547, 4.686]",4.686
4,HIM,Wicked Game,3.909333,DEC689800341,[4.6418333333333335],4.6418333333333335


In [67]:
dct_for_fill_gap_in_isrc = dict(zip(df_for_na_in_isrc['isrc'], 
                                    list(zip(df_for_na_in_isrc['title'].tolist(), df_for_na_in_isrc['eq_dur_from_meta'].tolist()))))

In [68]:
for k,v in tqdm(dct_for_fill_gap_in_isrc.items()):
    title_from_df_for_na_in_isrc = v[0]
    duration_from_df_for_na_in_isrc = v[1]
    if duration_from_df_for_na_in_isrc:
        meta.loc[(meta['title'] == title_from_df_for_na_in_isrc) & (meta['duration'] == duration_from_df_for_na_in_isrc), 'isrc'] = k
    else:
        meta.loc[(meta['title'] == title_from_df_for_na_in_isrc), 'isrc'] = k

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:01<00:00, 103.44it/s]


Посмотрим какие треки остались без isrc

In [69]:
meta[meta['isrc'].isna()]

,track_id,dttm,title,language,isrc,genres,duration,is_isrc
2151,16645e5e127d29b8bbbea6375a9344e2,2021-12-23 21:00:00,Глава 4. Урок рисования,None,None,FAIRYTALES,9.317333,0
2185,96f75d124808c84d823f09add1b6855c,2021-12-23 09:05:46,Чижик-пыжик (А. Пинегин — А. Усачев),None,None,FORCHILDREN,2.409333,0
2238,77bf68f7657ba9f87ee6ae7af3dc8999,2021-12-23 15:13:33,Повторение счёта до 12,None,None,FORCHILDREN,1.628667,0
2317,fb97e639240fb249756b40e0f88387cb,2021-12-24 08:21:48,Глава 12. Пятно,None,None,FAIRYTALES,1.721333,0
3838,be76b11b186c8f8a6afb34a23f64b132,2022-01-20 11:55:03,Americano,None,None,unknown,3.763333,0
3839,57592bb19732a5b36568b96dd077b9fd,2022-01-20 11:59:27,Снег идет,None,None,unknown,4.659333,0
4820,d241ca50664a10dbca4cf57a3c984cbe,2022-02-03 21:00:00,Круги на воде,None,None,RUSESTRADA ESTRADA,3.547000,0
8197,4ed5eabe40d2148c57e111c0b8903800,2022-03-23 07:10:36,Он тебя не любил,None,None,unknown,3.339667,0
9443,b4c2d3184013a5c459081829a090b6d8,2022-04-25 06:20:52,Bir kecha mehmoning bo'lay,None,None,POP UZBEKPOP,2.846833,0
9444,1a8bf8401636061cf2ca2a07565db7d3,2022-04-25 06:20:52,Dido anam dido,None,None,POP UZBEKPOP,3.541667,0


Есть композии, которые возможно не проходили регистрацию в крупных лейблах, или записывались на локальном уровне.

Или  возможно это композиция дубликат.

Либо по какой то причине запись для трека отсустствует в базе или было не корректно указано название.

Заполним пропуски значением "unknown".

In [70]:
meta.fillna(value={"isrc": "unknown"}, inplace=True)

В датасете так же есть строка с пропусками почти во всех значениях. Удалим ее.

In [71]:
meta.dropna(subset=['track_id', 'dttm', 'title', 'isrc', 'genres', 'duration'], inplace=True)

In [72]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71399 entries, 0 to 71768
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   track_id  71399 non-null  object        
 1   dttm      71399 non-null  datetime64[ns]
 2   title     71399 non-null  object        
 3   language  21802 non-null  object        
 4   isrc      71399 non-null  object        
 5   genres    71399 non-null  object        
 6   duration  71399 non-null  float64       
 7   is_isrc   71399 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 4.9+ MB


Переведем все тескстовые значения датасетов в нижний регистр

In [73]:
meta = meta.progress_apply(lambda x: x.astype(str).str.lower() if x.dtype == 'object' else x)
covers = covers.progress_apply(lambda x: x.astype(str).str.lower() if x.dtype == 'object' else x)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Для датасета covers, заменим бинарные текстовое значения для признака "track_remake_type" на цифровые для удобства

In [74]:
covers.replace({'track_remake_type': {'original': 1, 'cover': 0}}, inplace=True)

In [75]:
del df_for_na_in_isrc, get_isrc_for_track, find_close_num, k, v, title_from_df_for_na_in_isrc

In [76]:
del dct_for_fill_gap_in_isrc, duration_from_df_for_na_in_isrc

Выгрузим обработанные датасеты для дальнейшего использования

In [77]:
lyrics.to_csv(cwd+'/data_after/lyrics_after_first.csv')
meta.to_csv(cwd+'/data_after/meta_after_first.csv')
covers.to_csv(cwd+'/data_after/covers_after_first.csv')

# Отчет по этапу знакомства с данными и первичной предобработке.


        1. В датасете lyrics для id некоторых треков были обнаружены дубликаты текстов.
        Было проведено небольшое исследование на схожесть текстов каждого дубликата между собой.
        В качестве инструмента использовали меру Жаккара + n-граммы.
        Так некоторые тексты могли иметь не значительные отличия, но и полностью другой набор слов.
        Возможно данные загружали из разных источников. Из дубликатов текста было принято решение оставить наиболее 
        полные по количеству слов.
        
        Кроме дубликатов в датасете были обнаружены треки, для которых отсутствует разметка целевого признака,
        и они полностью не присутствуют в датасете covers. Такие треки тоже были удалены из набора.
        2. В датасете meta были: 
               2.1 Скорректированы значения для даты и продолжительности трека.
                   Во время обработки были обнаружены строки с небольшой длительность трека,
                   возможно это сэмпл или рингтон, в дальнейшем выделим их в отдельные группы.
               2.2 Были выявлены дубликаты в процессе заполнения пропусков в колонке isrc.
                   Одному треку могут соотвествовать несколько глобальных идентификаторов.
                   Как инструмент заполнения был выбран ресурс musicbrainz и библиотека request.
                   Кроме того, надежность источника выбранного для заполнения пропусков вызывает сомнение.
                   Принимая во внимание такие факты, возникает вопрос о корректности изначальных значений
                   в колонке и целесообразности ее дальнейшего использования.
               2.3 Колонка "язык песни" содержит большое количетсво пропусков.
                   На данном этапе оставили пропуски без изменения,
                   восстановим значения после того, как соберем единый датасет.
                   Значения будем восстанавливать по тексту песни.
               
        3. Все текстовые значения датасетов были преведены к нижнему регистру.